In [1]:
import pandas as pd
from datetime import datetime
import re

class AEDPoint:
    def __init__(self, id, x, y, address, indication, schedule, taken):
        self.id = id
        self.x = x
        self.y = y
        self.address = address
        self.indication = indication
        self.schedule = schedule
        self.taken = taken

class MapAEDS:
    def __init__(self):
        self.points = {}

    def add_point(self, point, availability):
        self.points[point.id] = [point, availability]
        print("Succesfully Added")

    def delete_point(self, point):
        if point in self.points:
            del self.points[point.id]
            print(f"Point with ID {point.id} deleted.")
        else:
            print("Point ID not found.")

    def update_availability(self, id, method, sit = None, time = None):
        
        if method == 1:
            if is_in_schedule(self.points[id][0].schedule, time) == True:
                self.points[id][1] = True
            else: 
                self.points[id][1] = False
        if method == 2: 
            if sit == 'take':
                self.points[id][1] = False
                self.points[id][0].taken = True 
            elif sit == 'return':
                self.points[id][1] = True
                self.points[id][0].taken = False 
                

        #Maybe add simulation of users elif method == 2: 
            

    def points_within_radius(self, user_loc, radius):
        # Initialize an empty list to store the points within the circle
        while radius > 0:
            points_within_circle = {}
            print(points_within_circle)
            found_points = 0  # Track the number of points found within the current radius
            for point in self.points:
                if self.points[point][1] == True:
                    # Get the longitude and latitude from the row
                    lat, lon = self.points[point][0].x, self.points[point][0].y
                    if lon < -90 or lon > 90 or lat < -90 or lat > 90:
                        continue
                    # Calculate the distance from the point to the center of the circle
                    distance = geodesic((lat, lon), user_loc).km
                    # If the distance is less than or equal to the radius, add the point to the list
                    if distance <= radius:
                        points_within_circle[id] = (lat, lon)
                        found_points += 1
                        if found_points == 24:
                            break
            if 0 < found_points <= 23:
                break  # If no points were found within the current radius, exit the loop
            elif found_points == 0:
                radius += 5 # Decrease radius
            else:
                radius -= 0.5
            i += 1
    
        return points_within_circle

In [2]:
import geocoder
import geopy.distance
import math
import pandas as pd
from geopy.distance import geodesic
def get_location():
    g = geocoder.ip('me')
    return g.latlng  # Return the coordinates

# Your location
user_loc = get_location()

In [3]:
def is_in_schedule(schedule, datetime_obj):
    # Get the day of the week from the datetime object
    day_of_week = datetime_obj.strftime("%A")

    # Check if the day is in the schedule
    if day_of_week not in schedule:
        return True 

    if schedule[day_of_week] == 0:
        return False

    # Get the intervals for the day from the schedule
    intervals = schedule[day_of_week]
    intervals = intervals.strip('[]').split(', ')
    print(intervals)
    intervals_fix = []
    for interval in intervals:
        splits = interval.split('-')
        interval_lst = []
        for element in splits:
            interval_lst.append(element.strip())
        intervals_fix.append(interval_lst)
    print(intervals_fix)

    for start, end in intervals_fix:
        pattern = r'^\d{2}'
    
        # Use the search function to find if the pattern matches the string
        match_start = re.search(pattern, start)
        match_end = re.search(pattern, end)
        if bool(match_start) == True:
            if int(start[:2]) > 12:
                new_num = int(start[:2]) - 12
                start = str(new_num) + start[2:]
        if bool(match_end) == True:
            if int(end[:2]) > 12:
                new_num = int(end[:2]) - 12
                end = str(new_num) + end[2:]
    
        try:
            start_time = datetime.strptime(start, '%I %p').time()
        except ValueError:
            start_time = datetime.strptime(start, '%I:%M %p').time()

        try:
            end_time = datetime.strptime(end, '%I %p').time()
            
        except ValueError:
            end_time = datetime.strptime(end, '%I:%M %p').time()        

        # Check if the datetime falls within this interval
        if start_time <= datetime_obj.time() <= end_time:
            return True

    # If none of the intervals match, return False
    return False

In [4]:
all_aeds = pd.read_excel("C:/Users/jorge/Downloads/aed_lat_long_availability (14).xlsx")
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]
start_index = 15227

# Select the last 1000 rows starting from the defined index until the end and swap latitude and longitude columns within that subset
last_1000_rows = all_aeds.iloc[start_index:]
last_1000_rows['latitude'], last_1000_rows['longitude'] = last_1000_rows['longitude'], last_1000_rows['latitude']

# Update the last 1000 rows in the original DataFrame
all_aeds.iloc[start_index:] = last_1000_rows

all_aeds = all_aeds.dropna(subset=['longitude'])

# Reset index
all_aeds = all_aeds.reset_index(drop=True)

for i in range(0, len(all_aeds)):
    for column in weekdays:
        if type(all_aeds[column][i]) == str:
            if all_aeds[column][i][-1] != ']':
                new_val = all_aeds[column][i].strip()
                all_aeds[column][i] = new_val

C:\Users\jorge\AppData\Local\Temp\ipykernel_25468\1557408453.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_1000_rows['latitude'], last_1000_rows['longitude'] = last_1000_rows['longitude'], last_1000_rows['latitude']
C:\Users\jorge\AppData\Local\Temp\ipykernel_25468\1557408453.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_aeds[column][i] = new_val


In [21]:
all_aeds

,Unnamed: 0.1,Unnamed: 0,id,address,province,location,location_aed,latitude,longitude,available,hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,0.0,13,blvd. fr. roosevelt,hainaut,NaN,"blvd. fr. roosevelt,7060.0,soignies,Belgium",50.579303,4.066629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1.0,70,ch. de wegimont,liege,NaN,"ch. de wegimont,4630.0,ayeneux,Belgium",50.612395,5.737884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.0,71,place saint - lambert,liege,NaN,"place saint - lambert,4020.0,liege,Belgium",50.645622,5.573620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3.0,72,rue du doyard,liege,NaN,"rue du doyard,4990.0,lierneux,Belgium",50.287416,5.786325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4.0,73,fond saint servais,liege,NaN,"fond saint servais,4000.0,liege,Belgium",50.646806,5.571031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16259,16265,NaN,NaN,NaN,NaN,NaN,NaN,50.770352,3.876796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16260,16266,NaN,NaN,NaN,NaN,NaN,NaN,50.932726,4.043508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16261,16267,NaN,NaN,NaN,NaN,NaN,NaN,51.046250,4.171364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16262,16268,NaN,NaN,NaN,NaN,NaN,NaN,51.074329,3.716076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
all_aeds = all_aeds.drop(columns = ["Unnamed: 0.1", "Unnamed: 0"])

In [25]:
all_aeds

,id,address,province,location,location_aed,latitude,longitude,available,hours,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,13,blvd. fr. roosevelt,hainaut,NaN,"blvd. fr. roosevelt,7060.0,soignies,Belgium",50.579303,4.066629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,70,ch. de wegimont,liege,NaN,"ch. de wegimont,4630.0,ayeneux,Belgium",50.612395,5.737884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71,place saint - lambert,liege,NaN,"place saint - lambert,4020.0,liege,Belgium",50.645622,5.573620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,72,rue du doyard,liege,NaN,"rue du doyard,4990.0,lierneux,Belgium",50.287416,5.786325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,73,fond saint servais,liege,NaN,"fond saint servais,4000.0,liege,Belgium",50.646806,5.571031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16259,NaN,NaN,NaN,NaN,NaN,50.770352,3.876796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16260,NaN,NaN,NaN,NaN,NaN,50.932726,4.043508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16261,NaN,NaN,NaN,NaN,NaN,51.046250,4.171364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16262,NaN,NaN,NaN,NaN,NaN,51.074329,3.716076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
all_aeds.to_csv("All_Aeds.csv")

In [57]:
all_aeds.loc[14041]

Unnamed: 0.1                                                14047
Unnamed: 0                                                14047.0
id                                                           8086
address                                  kleine karmelietenstraat
province                                          oost-vlaanderen
location                             Gang ingang aan secretariaat
location_aed    kleine karmelietenstraat,9500.0,geraardsbergen...
latitude                                                50.774663
longitude                                                3.876959
available                                                 Non-Nee
hours           Maandag, dinsdag, donderdag en vrijdag 08.00 u...
Monday                                           [8 AM - 4:30 PM]
Tuesday                                          [8 AM - 4:30 PM]
Wednesday                                           [8 AM -12 PM]
Thursday                                         [8 AM - 4:30 PM]
Friday    

In [58]:
# Example usage:
maps = MapAEDS()
for i in range(0, len(all_aeds)):
    dict_schedule = {}
    for weekday in weekdays:
        if isinstance(all_aeds[weekday][i] , str) or all_aeds[weekday][i] == 0:
            dict_schedule[weekday] = all_aeds[weekday][i]            
    point = AEDPoint(i, all_aeds['latitude'][i], all_aeds['longitude'][i], all_aeds['address'][i], all_aeds['location'][i], dict_schedule, False)
    maps.add_point(point, True)

Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfully Added
Succesfull

In [ ]:
current_time = datetime.now()
i = 0
for point in maps.points:
    if maps.points[point][0].taken == False:
        maps.update_availability(point, 1, time = current_time)
        print(i)
    i += 1

In [17]:
datetime.strptime('12 PM', '%I %p').time()

datetime.time(12, 0)

In [65]:
import re
pattern = r'^\d{2}'

start = '20:30 PM'
# Use the search function to find if the pattern matches the string
match_start = re.search(pattern, start)
if bool(match_start) == True:
    if int(start[:2]) > 12:
        new_num = int(start[:2]) - 12
        start = str(new_num) + start[2:]

In [66]:
start

'8:30 PM'

In [9]:
datetime.strptime('9:30 PM', '%I %p').time()

time_string = '9:30 PM'
time_format = '%I:%M %p'

try:
    time_object = datetime.strptime(time_string, time_format)
    print("Time successfully parsed:", time_object.time())
except ValueError:
    print("Error: Time data does not match the expected format.")

ValueError: time data '9:30 PM' does not match format '%I %p'

In [11]:
import re

# Input string
time_string = '[6 AM - 1:30 PM, 3 PM - 5 PM]'

# Remove brackets and split by comma
intervals = time_string.strip('[]').split(', ')

result = []
for interval in intervals:
    # Extract start and end times
    start, end = re.findall(r'\b\d{1,2}:\d{2} [AP]M\b', interval)
    
    # Append the extracted times to the result list
    result.append((start, end))

print(result)

ValueError: not enough values to unpack (expected 2, got 1)

In [31]:
# Input string
time_string = '[6 AM - 1:30 PM, 3 PM - 5 PM]'

# Remove brackets and split by comma
intervals = time_string.strip('[]').split(', ')

In [32]:
intervals

['6 AM - 1:30 PM', '3 PM - 5 PM']

In [35]:
start_time = datetime.strptime(intervals_fix[0][0], '%I %p').time()


In [36]:
start_time

datetime.time(6, 0)

In [34]:
intervals_fix

[['6 AM', '1:30 PM'], ['3 PM', '5 PM']]

In [ ]:
re.findall(r'\b\d{1,2}:\d{2} [AP]M\b', interval)


In [ ]:
parse_time_range(time_range)